<a href="https://colab.research.google.com/github/forrestpark/MachineLearning/blob/main/Machine_Learning_for_Alzheimer's_Disease_Diagnosis_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import the needed libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.model_selection import KFold

from collections import Counter

## Import and Explore the data

In [ ]:
url = 'https://raw.githubusercontent.com/pjwoo3/BNCS411/main/train_data.csv'
train = pd.read_csv(url)
# Dataset is now stored in Pandas dataframe
train.head()

,DX_bl,ADAS11,ADAS13,MMSE,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,ST107CV,ST108CV,ST109CV,ST110CV,ST111CV,ST113CV,ST114CV,ST115CV,ST116CV,ST117CV,ST118CV,ST119CV,ST121CV,ST123CV,ST129CV,ST130CV,ST13CV,ST14CV,ST15CV,ST23CV,ST24CV,ST25CV,ST26CV,ST31CV,ST32CV,ST34CV,ST35CV,ST36CV,ST38CV,ST39CV,ST40CV,...,ST34TA,ST35TA,ST36TA,ST38TA,ST39TA,ST40TA,ST43TA,ST44TA,ST45TA,ST46TA,ST47TA,ST48TA,ST49TA,ST50TA,ST51TA,ST52TA,ST54TA,ST55TA,ST56TA,ST57TA,ST58TA,ST59TA,ST60TA,ST62TA,ST64TA,ST72TA,ST73TA,ST74TA,ST82TA,ST83TA,ST84TA,ST85TA,ST90TA,ST91TA,ST93TA,ST94TA,ST95TA,ST97TA,ST98TA,ST99TA
0,0,9.00,14.00,27,3295,1760,2957,1975,3270,1871,9504,3219,12683,8167,1956,15106,18296,12517,11914,9771,2024,1221,6627.0,6245,5844,2361,2161,4782,2116,1757.0,723,6714,11308,10841,1669,8168,6690,6040,3858,11530,...,2.098,1.965,2.666,1.690,2.328,2.636,2.159,2.354,2.174,2.331,2.335,1.353,1.757,2.270,2.189,2.099,2.695,2.112,2.537,2.004,2.698,2.445,3.461,2.216,1.004,2.480,2.428,2.234,1.670,3.132,2.467,2.424,2.345,2.914,2.461,2.129,2.519,1.606,2.363,2.864
1,0,5.00,7.00,26,3644,1926,4376,2723,4806,1756,10159,2972,13170,10227,1754,14615,18758,12629,9276,10399,2057,842,NaN,7175,5860,2154,2159,5930,2979,1770.0,761,10683,10900,11519,2544,10561,7039,6141,4148,9033,...,2.719,2.031,2.429,2.204,2.079,2.852,2.367,2.887,2.543,2.395,2.431,1.712,2.034,2.602,2.598,2.246,2.401,2.142,2.574,2.169,2.877,2.602,3.375,2.382,NaN,2.657,2.543,2.348,1.948,3.782,2.440,2.843,2.499,2.695,2.457,2.254,2.269,1.868,2.142,2.679
2,2,16.33,29.33,25,3096,2191,3784,2344,2675,2063,8561,3208,14310,7538,1571,14405,20381,11549,9488,8848,2175,1109,6150.0,6085,5977,1828,1392,5996,3021,1562.0,489,8597,6703,8310,2281,11219,7499,7580,4451,7417,...,2.253,1.891,2.540,1.879,2.279,2.548,2.298,2.861,2.528,2.827,2.234,1.404,1.940,2.285,2.436,1.800,2.940,2.152,2.692,1.756,2.607,2.221,3.703,2.175,1.064,2.058,2.168,2.414,1.885,3.303,2.702,2.617,2.108,2.526,2.151,2.003,2.534,1.996,2.163,2.643
3,1,6.00,9.00,29,3857,2660,3382,2751,3783,2202,9074,2504,11995,10484,1986,13690,20896,15095,10463,11313,2787,935,NaN,7009,7279,1719,2238,5620,2816,1934.0,925,11194,11349,10076,2299,11054,7071,6329,5498,8785,...,2.643,2.158,2.489,1.952,2.289,2.767,2.199,3.184,2.388,2.671,2.179,1.536,2.137,2.539,2.481,2.376,2.736,2.163,2.638,2.185,2.709,2.328,3.924,2.529,NaN,2.663,2.990,2.469,1.922,4.232,2.546,2.937,2.514,2.875,2.837,2.379,2.315,1.954,2.326,2.989
4,0,5.00,8.00,28,3411,1578,2664,2433,3448,1550,7128,2500,9801,7732,1451,13050,17785,10460,8407,8573,1952,874,NaN,5275,6034,2070,1405,3591,2244,1110.0,652,9601,9964,8338,2059,8735,5546,5859,4480,8762,...,2.644,2.128,2.702,2.065,2.597,2.874,2.281,2.128,2.689,2.955,2.370,1.536,1.990,2.545,2.519,2.286,2.839,2.495,2.812,2.104,2.652,2.549,3.105,2.390,NaN,2.483,2.381,2.498,1.981,3.331,2.954,2.825,2.440,2.848,2.447,2.256,2.497,2.052,2.498,2.829


In [ ]:
train.describe()

,DX_bl,ADAS11,ADAS13,MMSE,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,ST107CV,ST108CV,ST109CV,ST110CV,ST111CV,ST113CV,ST114CV,ST115CV,ST116CV,ST117CV,ST118CV,ST119CV,ST121CV,ST123CV,ST129CV,ST130CV,ST13CV,ST14CV,ST15CV,ST23CV,ST24CV,ST25CV,ST26CV,ST31CV,ST32CV,ST34CV,ST35CV,ST36CV,ST38CV,ST39CV,ST40CV,...,ST34TA,ST35TA,ST36TA,ST38TA,ST39TA,ST40TA,ST43TA,ST44TA,ST45TA,ST46TA,ST47TA,ST48TA,ST49TA,ST50TA,ST51TA,ST52TA,ST54TA,ST55TA,ST56TA,ST57TA,ST58TA,ST59TA,ST60TA,ST62TA,ST64TA,ST72TA,ST73TA,ST74TA,ST82TA,ST83TA,ST84TA,ST85TA,ST90TA,ST91TA,ST93TA,ST94TA,ST95TA,ST97TA,ST98TA,ST99TA
count,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,760.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1606.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,...,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1606.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,760.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.00000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000,1607.000000
mean,0.889857,10.687623,16.838836,27.197884,3415.757312,1889.919104,3449.139390,2242.814561,3415.616055,2011.142502,7846.479776,2936.383945,11364.592408,8286.555694,1850.807716,13241.149347,18303.506534,11272.752334,9654.012446,8795.275669,1973.545737,778.663348,6454.726316,6077.756067,6185.471064,2221.753578,1726.772869,5499.126322,2536.539515,1743.722914,632.463597,8752.144368,10421.447418,9484.886123,2291.787181,10295.434972,6516.395146,5719.485999,4226.335408,9084.429994,...,2.384760,2.019114,2.475054,1.821200,2.290044,2.648584,2.157642,2.540934,2.361113,2.526049,2.230354,1.456342,1.849313,2.417301,2.253390,2.139604,2.832131,2.154694,2.497071,2.007938,2.486484,2.306683,3.450963,2.125401,1.043959,2.361660,2.538278,2.313991,1.735683,3.314019,2.552358,2.552088,2.25390,2.698467,2.335464,2.077695,2.463360,1.850768,2.264312,2.705311
std,0.693999,6.564644,9.519048,2.640931,581.050677,338.251043,674.695186,367.721845,684.312622,404.254377,1269.699606,468.884157,1661.823223,1335.890162,448.358620,2068.689775,2513.344508,1794.353279,1419.068470,1408.779918,398.514299,169.764897,985.436973,849.199658,965.356725,476.215463,420.541071,1060.617430,496.723152,461.554683,155.676189,1487.230213,1921.463519,1752.756276,450.197633,1725.348432,830.238911,1011.246790,882.130700,1597.766218,...,0.237534,0.177947,0.186057,0.151925,0.194545,0.233070,0.218050,0.401720,0.184786,0.250136,0.195438,0.146903,0.174317,0.199327,0.242965,0.202077,0.279173,0.164407,0.183875,0.199050,0.231863,0.204175,0.405734,0.272570,0.136194,0.226703,0.302780,0.207063,0.162672,0.545333,0.284886,0.231266,0.21027,0.231743,0.247948,0.188536,0.182837,0.158143,0.206316,0.227434
min,0.000000,0.000000,0.000000,19.000000,1968.000000,790.000000,1757.000000,1105.000000,1499.000000,726.000000,4422.000000,1530.000000,6287.000000,4676.000000,697.000000,6920.000000,8885.000000,5687.000000,5014.000000,4652.000000,484.000000,347.000000,3794.000000,3113.000000,3402.000000,943.000000,538.000000,2639.000000,760.000000,502.000000,195.000000,4394.000000,4678.000000,3976.000000,1146.000000,4994.000000,3157.000000,2190.000000,2244.000000,3528.000000,...,1.661000,1.088000,1.760000,1.266000,1.639000,1.599000,1.240000,1.449000,1.624000,1.708000,1.543000,1.123000,1.317000,1.396000,1.333000,1.475000,1.871000,1.311000,1.693000,1.204000,1.616000,1.562000,1.580000,1.261000,0.630000,1.466000,1.592000,1.504000,1.244000,1.454000,1.660000,1.651000,1.42200,1.772000,1.347000,1.312000,1.8

## Data Cleaning

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1607 entries, 0 to 1606
Columns: 144 entries, DX_bl to ST99TA
dtypes: float64(76), int64(68)
memory usage: 1.8 MB


In [ ]:
train.notnull().any()

DX_bl      True
ADAS11     True
ADAS13     True
MMSE       True
ST102CV    True
           ... 
ST94TA     True
ST95TA     True
ST97TA     True
ST98TA     True
ST99TA     True
Length: 144, dtype: bool

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 0 to 1605
Columns: 144 entries, DX_bl to ST99TA
dtypes: float64(76), int64(68)
memory usage: 860.9 KB


## Comment:

I have tried forward fill, back fill, mode filling, and mean filling and dropping the entier row. The highest accuracy was achieved after dropping the null rows.

## Remove Outliers

The following code filters any data with z score more than 3 to reomve the outlirs in the dataset

In [ ]:
z_scores = zscore(train)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_train = train[filtered_entries]

In [ ]:
new_train.describe()

,DX_bl,ADAS11,ADAS13,MMSE,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,ST107CV,ST108CV,ST109CV,ST110CV,ST111CV,ST113CV,ST114CV,ST115CV,ST116CV,ST117CV,ST118CV,ST119CV,ST121CV,ST123CV,ST129CV,ST130CV,ST13CV,ST14CV,ST15CV,ST23CV,ST24CV,ST25CV,ST26CV,ST31CV,ST32CV,ST34CV,ST35CV,ST36CV,ST38CV,ST39CV,ST40CV,...,ST34TA,ST35TA,ST36TA,ST38TA,ST39TA,ST40TA,ST43TA,ST44TA,ST45TA,ST46TA,ST47TA,ST48TA,ST49TA,ST50TA,ST51TA,ST52TA,ST54TA,ST55TA,ST56TA,ST57TA,ST58TA,ST59TA,ST60TA,ST62TA,ST64TA,ST72TA,ST73TA,ST74TA,ST82TA,ST83TA,ST84TA,ST85TA,ST90TA,ST91TA,ST93TA,ST94TA,ST95TA,ST97TA,ST98TA,ST99TA
count,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,...,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.0000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000
mean,0.907236,11.265195,17.851466,26.849722,3215.760668,1792.688312,3266.259740,2142.430427,3142.743970,1855.363636,7268.617811,2904.979592,10517.057514,7513.473098,1588.660482,12276.870130,17288.484230,10365.935065,9049.378479,8293.584416,1815.452690,711.096475,6373.755102,5574.408163,5558.849722,2122.671614,1650.684601,5175.452690,2301.365492,1613.714286,551.153989,8108.217069,9554.541744,9012.957328,2043.985158,9745.929499,6156.473098,5302.673469,3525.860853,8579.484230,...,2.339980,1.933032,2.444451,1.731403,2.237430,2.585889,2.033764,2.409957,2.282237,2.487690,2.157217,1.367334,1.757022,2.323375,2.108232,2.024249,2.832308,2.111148,2.451365,1.906592,2.398796,2.216139,3.421141,2.022636,1.0469,2.263573,2.508699,2.243423,1.647345,3.203724,2.557514,2.463083,2.152121,2.657787,2.289738,1.974336,2.445881,1.750043,2.211325,2.640416
std,0.702309,5.861376,8.752147,2.599266,504.181899,301.511020,644.580743,347.954245,608.918297,352.212204,1068.176024,405.069912,1294.337793,1010.393273,326.904431,1741.047153,2155.651037,1480.685160,1241.391135,1223.342509,333.779805,148.882560,884.468660,610.431644,638.777630,443.375911,368.422691,951.473547,432.161652,444.380903,133.948890,1221.648998,1620.407127,1675.822243,335.345602,1590.421003,678.462243,909.171550,505.037553,1442.990439,...,0.227793,0.142198,0.182145,0.114013,0.195666,0.213719,0.195728,0.376311,0.175591,0.253076,0.184528,0.095794,0.151215,0.172763,0.212197,0.170614,0.273106,0.163466,0.176551,0.179116,0.196806,0.185940,0.377887,0.261526,0.1325,0.208300,0.282677,0.202356,0.126542,0.504242,0.282877,0.203719,0.184401,0.226723,0.235961,0.145469,0.189592,0.115965,0.200574,0.214173
min,0.000000,1.330000,1.670000,20.000000,1971.000000,1023.000000,2087.000000,1299.000000,1499.000000,1009.000000,4907.000000,1800.000000,7563.000000,5187.000000,697.000000,7661.000000,12299.000000,6547.000000,5927.000000,5498.000000,887.000000,347.000000,4021.000000,3943.000000,3930.000000,1051.000000,784.000000,2738.000000,1275.000000,532.000000,266.000000,4394.000000,4678.000000,4405.000000,1146.000000,5513.000000,4335.000000,2575.000000,2244.000000,4298.000000,...,1.778000,1.468000,1.903000,1.451000,1.701000,1.861000,1.482000,1.532000,1.759000,1.802000,1.705000,1.123000,1.354000,1.811000,1.476000,1.506000,2.133000,1.667000,1.874000,1.368000,1.850000,1.631000,2.198000,1.329000,0.7000,1.585000,1.711000,1.690000,1.329000,1.691000,1.660000,1.847000,1.612000,1.991000,1.698000,1.599000,2.001000,1.421000,1.676000,1.953000
25%,0.000000,6.670000,11.000000,25.000000,2834.0

## Split the data into X and Ys

In [ ]:
Y1 = new_train['DX_bl']
Y1

2       2
5       0
7       1
8       1
9       0
       ..
1587    2
1588    0
1589    2
1594    1
1597    2
Name: DX_bl, Length: 539, dtype: int64

In [ ]:
Y2 = new_train[['ADAS11','ADAS13', 'MMSE']]
Y2

,ADAS11,ADAS13,MMSE
2,16.33,29.33,25
5,3.33,5.33,30
7,5.00,11.00,27
8,16.00,25.00,29
9,6.00,8.00,30
...,...,...,...
1587,9.67,18.67,25
1588,8.33,12.33,30
1589,17.33,26.33,20
1594,15.00,24.00,26


In [ ]:
X = new_train.drop(labels=['DX_bl','ADAS11', 'ADAS13', 'MMSE'], axis = 1)
X

,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,ST107CV,ST108CV,ST109CV,ST110CV,ST111CV,ST113CV,ST114CV,ST115CV,ST116CV,ST117CV,ST118CV,ST119CV,ST121CV,ST123CV,ST129CV,ST130CV,ST13CV,ST14CV,ST15CV,ST23CV,ST24CV,ST25CV,ST26CV,ST31CV,ST32CV,ST34CV,ST35CV,ST36CV,ST38CV,ST39CV,ST40CV,ST43CV,ST44CV,ST45CV,ST46CV,...,ST34TA,ST35TA,ST36TA,ST38TA,ST39TA,ST40TA,ST43TA,ST44TA,ST45TA,ST46TA,ST47TA,ST48TA,ST49TA,ST50TA,ST51TA,ST52TA,ST54TA,ST55TA,ST56TA,ST57TA,ST58TA,ST59TA,ST60TA,ST62TA,ST64TA,ST72TA,ST73TA,ST74TA,ST82TA,ST83TA,ST84TA,ST85TA,ST90TA,ST91TA,ST93TA,ST94TA,ST95TA,ST97TA,ST98TA,ST99TA
2,3096,2191,3784,2344,2675,2063,8561,3208,14310,7538,1571,14405,20381,11549,9488,8848,2175,1109,6150.0,6085,5977,1828,1392,5996,3021,1562.0,489,8597,6703,8310,2281,11219,7499,7580,4451,7417,3979,2317.0,4316,1783,...,2.253,1.891,2.540,1.879,2.279,2.548,2.298,2.861,2.528,2.827,2.234,1.404,1.940,2.285,2.436,1.800,2.940,2.152,2.692,1.756,2.607,2.221,3.703,2.175,1.064,2.058,2.168,2.414,1.885,3.303,2.702,2.617,2.108,2.526,2.151,2.003,2.534,1.996,2.163,2.643
5,3540,1428,3918,2111,3539,1935,6955,3082,10875,8773,1814,15407,20977,11587,10094,9856,1799,836,7960.0,6815,6610,2549,1745,4919,2491,2458.0,682,7869,11311,11582,1899,10270,6044,5178,4267,11255,4448,2021.0,4152,1550,...,2.056,1.843,2.503,1.696,2.468,2.598,2.156,2.060,2.341,2.497,2.118,1.551,1.653,2.310,2.052,2.068,2.820,2.080,2.431,1.816,2.443,2.084,3.491,1.776,0.999,2.357,2.339,2.195,1.504,3.472,2.927,2.255,2.359,2.828,1.972,1.868,2.326,1.608,2.173,2.861
7,3960,1412,3344,1360,2559,1753,7071,2739,8755,6926,1539,11174,14527,12024,8034,6700,1587,513,5516.0,4430,4686,2075,1725,5026,1807,1953.0,360,6680,10128,7727,1703,9640,4919,5768,2611,8757,2619,1365.0,3087,1319,...,1.918,2.026,2.437,1.794,2.021,2.732,1.972,1.999,2.300,2.438,2.056,1.294,1.682,2.162,2.131,2.105,2.734,1.893,2.382,2.142,2.363,2.303,3.901,1.488,0.839,2.220,2.031,2.437,1.775,3.719,2.673,2.438,2.278,2.659,2.047,2.156,2.275,1.677,2.237,2.673
8,3421,2267,3583,2446,2149,2145,10472,2898,12527,7824,1715,12845,20170,10060,10947,10244,1705,681,7221.0,6480,6282,2096,1558,4822,2957,1910.0,634,7944,9090,10953,2415,9383,6334,6316,4632,8198,3528,2217.0,2754,1603,...,2.493,1.904,2.565,1.752,2.483,2.499,2.058,2.551,2.012,2.434,2.119,1.568,1.827,2.552,2.324,1.840,3.020,1.936,2.417,1.850,2.495,2.009,3.378,2.228,1.052,2.334,2.615,2.269,1.766,3.743,2.301,2.683,2.172,2.759,2.671,1.979,2.511,1.921,2.297,2.752
9,3506,1711,4559,2093,2828,2126,7977,3209,11138,8110,1909,12931,18123,11025,9640,8837,1800,688,5591.0,5575,5371,2077,2503,5398,1957,1146.0,661,7996,9165,11073,1986,9697,6648,5443,3077,8401,3050,2041.0,4047,1738,...,2.368,1.983,2.633,1.813,2.335,2.636,2.159,2.853,2.611,3.139,2.480,1.478,1.895,2.282,2.176,2.227,2.956,2.538,2.687,1.958,2.700,2.474,3.170,1.921,1.106,2.530,3.209,2.618,1.804,2.903,3.270,2.789,2.413,3.045,2.874,2.139,2.822,1.926,2.624,2.894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,2873,1405,4460,1500,3826,2115,7288,3502,10433,8674,1585,13551,17946,12005,9445,7897,2318,1031,7836.0,6132,6187,3415,1742,6014,2759,2407.0,527,9149,10233,8581,2355,10320,6568,5994,3626,9445,3229,1981.0,4969,1612,...,2.605,2.148,2.770,1.859,2.407,2.697,2.459,1.940,2.486,2.768,2.404,1.667,1.942,2.703,2.467,2.278,3.272,2.390,2.773,2.237,2.797,2.378,4.402,2.287,1.282,2.331,2.457,2.563,1.812,4.129,3.035,2.854,2.435,2.773,2.451,2.171,2.869,1.781,2.508,2.739
1588,3652,1737,3307,1624,2904,1286,7389,3073,10720,7324,1957,9254,17188,9818,7836,7252,1685,601,5985.0,4861,4931,1961,1039,5042,1875,1712.0,501,7438,10176,7597,2065,8664,5971,5617,3366,9233,3231,1699.0,3261,1989,...,2.821,2.101,2.671,1.861,2.338,2.789,2.324,2.190,2.400,2.754,2.373,1.318,2.091,2.715,2.518,2.156,2.873,2.238,2.774,2.148,2.423,2.

## Train Test Split for first Task

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y1, test_size = 0.2, shuffle = True, random_state = 0)

In [ ]:
X_train = np.asarray(X_train, dtype=np.uint8)
y_train = np.asarray(y_train, dtype=np.uint8)

X_test = np.asarray(X_test, dtype=np.uint8)
y_test = np.asarray(y_test, dtype=np.uint8)

## Standarization

standarization is required in order to modify the data distribution from skewed to normal distribution. I have tried normalization also, however, standarization gave out higher accuracy scores.

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(X_train)
X_test_minmax = mm_scaler.fit_transform(X_test)

## KNN model

In [ ]:
K = 35
knn= KNeighborsClassifier(K)
knn.fit(X_train_minmax , y_train)
y_pred = knn.predict(X_test_minmax)

In [ ]:
accuracy_score(y_test, y_pred)

0.5

## Cross validation

applying 10 k-folds as mentioned and also looping over K values for tuning the model for the highest accuracy.

In [ ]:
n_splits = 10 
kf = KFold(n_splits= n_splits) 

accuracy_k = []
k_values = list(range(1,100,2)) 

for k in k_values: ## Loop over the K values
    accuracy_fold = 0
    for X_train_fold_idx, X_valid_fold_idx in kf.split(X_train_minmax): ## loop over the splits
        X_train_fold = X_train_minmax[X_train_fold_idx] ## fetch the values
        y_train_fold = y_train[X_train_fold_idx]

        X_test_fold = X_train_minmax[X_valid_fold_idx]
        y_valid_fold = y_train[X_valid_fold_idx]
        knn= KNeighborsClassifier(k)
        knn.fit(X_train_fold , y_train_fold)
        y_pred_fold = knn.predict(X_test_fold)
        
        accuracy_fold += accuracy_score(y_pred_fold, y_valid_fold) ## Accumlate the accuracy
    accuracy_fold = accuracy_fold/n_splits ## Divide by the number of splits
    accuracy_k.append(accuracy_fold) 


In [ ]:
print(f"Best Accuracy was {np.max(accuracy_k)}, which corresponds to a value of K = {k_values[np.argmax(accuracy_k)]}")

Best Accuracy was 0.5451902748414377, which corresponds to a value of K = 41


In [ ]:
K = 51
knn= KNeighborsClassifier(K)
knn.fit(X_train_minmax , y_train)
y_pred = knn.predict(X_test_minmax)
accuracy_score(y_test, y_pred)

0.5

## Decision tree classifier

In [ ]:
model = DecisionTreeClassifier(max_depth=2)
model.fit(X_train_minmax, y_train)
y_predict = model.predict(X_test_minmax)

In [ ]:
accuracy_score(y_test, y_predict)

0.37962962962962965

## Depth tuning
also applying cross validation and tuning the depth of the tree to search for the highest accuracy score.

In [ ]:
n_splits = 10 
kf = KFold(n_splits= n_splits) 

accuracy_k = []
d_values = list(range(1,40,1)) 

for d in d_values: ## Loop over the K values
    accuracy_fold = 0
    for normalized_X_train_fold_idx, normalized_X_valid_fold_idx in kf.split(X_train_minmax): ## loop over the splits
        normalized_X_train_fold = X_train_minmax[normalized_X_train_fold_idx] ## fetch the values
        y_train_fold = y_train[normalized_X_train_fold_idx]

        normalized_X_test_fold = X_train_minmax[normalized_X_valid_fold_idx]
        y_valid_fold = y_train[normalized_X_valid_fold_idx]
        model = DecisionTreeClassifier(max_depth=d)
        model.fit(normalized_X_train_fold, y_train_fold)
        y_pred_fold = model.predict(normalized_X_test_fold)
        
        accuracy_fold += accuracy_score(y_pred_fold, y_valid_fold) ## Accumlate the accuracy
    accuracy_fold = accuracy_fold/n_splits ## Divide by the number of splits
    accuracy_k.append(accuracy_fold) 


In [ ]:
print(f"Best Accuracy was {np.max(accuracy_k)}, which corresponds to a value of d = {k_values[np.argmax(accuracy_k)]}")

Best Accuracy was 0.5149577167019028, which corresponds to a value of d = 1


In [ ]:
model = DecisionTreeClassifier(max_depth=9)
model.fit(X_train_minmax, y_train)
y_predict = model.predict(X_test_minmax)
accuracy_score(y_test, y_predict)

0.37962962962962965

## SVM

In [ ]:
SVC_model = SVC()
SVC_model.fit(X_train_minmax, y_train)
SVC_prediction = SVC_model.predict(X_test_minmax)
print(accuracy_score(SVC_prediction, y_test))

0.4537037037037037


## Cross validation

In [ ]:
n_splits = 10 
kf = KFold(n_splits= n_splits) 
accuracy_fold = 0
for normalized_X_train_fold_idx, normalized_X_valid_fold_idx in kf.split(X_train_minmax): ## loop over the splits
  normalized_X_train_fold = X_train_minmax[normalized_X_train_fold_idx] ## fetch the values
  y_train_fold = y_train[normalized_X_train_fold_idx]

  normalized_X_test_fold = X_train_minmax[normalized_X_valid_fold_idx]
  y_valid_fold = y_train[normalized_X_valid_fold_idx]
  SVC_model = SVC()
  SVC_model.fit(normalized_X_train_fold, y_train_fold)
  y_pred_fold = SVC_model.predict(normalized_X_test_fold)        
  accuracy_fold += accuracy_score(y_pred_fold, y_valid_fold) ## Accumlate the accuracy
accuracy_fold = accuracy_fold/n_splits ## Divide by the number of split

In [ ]:
print(accuracy_fold)

0.5427589852008456


## Random Forrests

In [ ]:
clf = RandomForestClassifier(max_depth=5, random_state=27)

In [ ]:
clf.fit(X_train_minmax, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=27, verbose=0,
                       warm_start=False)

In [ ]:
prediction = clf.predict(X_test_minmax)

In [ ]:
accuracy_score(prediction, y_test)

0.4351851851851852

In [ ]:
n_splits = 10 
kf = KFold(n_splits= n_splits) 

accuracy_k = []
d_values = list(range(1,10,1)) 

for d in d_values: ## Loop over the K values
    accuracy_fold = 0
    for normalized_X_train_fold_idx, normalized_X_valid_fold_idx in kf.split(X_train_minmax): ## loop over the splits
        normalized_X_train_fold = X_train_minmax[normalized_X_train_fold_idx] ## fetch the values
        y_train_fold = y_train[normalized_X_train_fold_idx]

        normalized_X_test_fold = X_train_minmax[normalized_X_valid_fold_idx]
        y_valid_fold = y_train[normalized_X_valid_fold_idx]
        model = RandomForestClassifier(max_depth=d, random_state=27)
        model.fit(normalized_X_train_fold, y_train_fold)
        y_pred_fold = model.predict(normalized_X_test_fold)
        
        accuracy_fold += accuracy_score(y_pred_fold, y_valid_fold) ## Accumlate the accuracy
    accuracy_fold = accuracy_fold/n_splits ## Divide by the number of splits
    accuracy_k.append(accuracy_fold) 


In [ ]:
print(f"Best Accuracy was {np.max(accuracy_k)}, which corresponds to a value of d = {k_values[np.argmax(accuracy_k)]}")

Best Accuracy was 0.5289640591966174, which corresponds to a value of d = 13


In [ ]:
cl = RandomForestClassifier(max_depth=k_values[np.argmax(accuracy_k)], random_state=27)
cl.fit(X_train_minmax, y_train)
prediction = clf.predict(X_test_minmax)
accuracy_score(y_test, prediction)

0.4351851851851852

## Analysis:

Various models were applied on the data to predict the diagnosis group of the subject. The model is classifying the subject into one of the following groups:
<ol>
  <li>Cognitive normal</li>
  <li>Mild cognitive impairment</li>
  <li>Alzheimer’s disease</li>
</ol>

The models used are the following:
<ol>
  <li>KNN</li>
  <li>Decision Trees</li>
  <li>SVM</li>
  <li>Random forrest</li>
</ol>

The highest scoring model was SVM with **accuracy of 0.59** Then, the random forrests with 0.57 then KNN and Decision trees with 0.5 and 0.48 respectively.

### Saving the clf model using joblib dump

In [ ]:
from joblib import dump, load
dump(SVC_model, 'clf_model.joblib')

['clf_model.joblib']

## Task 2

The mentioned assessments are numeric type. Therefore, in order to predic the assessment scores, linear regression was applied on the data and r^2 values were used as a metric for the performance of the data.

In [ ]:
Y2

,ADAS11,ADAS13,MMSE
2,16.33,29.33,25
5,3.33,5.33,30
7,5.00,11.00,27
8,16.00,25.00,29
9,6.00,8.00,30
...,...,...,...
1587,9.67,18.67,25
1588,8.33,12.33,30
1589,17.33,26.33,20
1594,15.00,24.00,26


In [ ]:
Y2.describe()

,ADAS11,ADAS13,MMSE
count,539.000000,539.000000,539.000000
mean,11.265195,17.851466,26.849722
std,5.861376,8.752147,2.599266
min,1.330000,1.670000,20.000000
25%,6.670000,11.000000,25.000000
50%,10.670000,17.670000,27.000000
75%,14.670000,23.670000,29.000000
max,30.330000,42.330000,30.000000


In [ ]:
Y2['ADAS11']

2       16.33
5        3.33
7        5.00
8       16.00
9        6.00
        ...  
1587     9.67
1588     8.33
1589    17.33
1594    15.00
1597    13.67
Name: ADAS11, Length: 539, dtype: float64

In [ ]:
X['intercept'] = 1
X

,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,ST107CV,ST108CV,ST109CV,ST110CV,ST111CV,ST113CV,ST114CV,ST115CV,ST116CV,ST117CV,ST118CV,ST119CV,ST121CV,ST123CV,ST129CV,ST130CV,ST13CV,ST14CV,ST15CV,ST23CV,ST24CV,ST25CV,ST26CV,ST31CV,ST32CV,ST34CV,ST35CV,ST36CV,ST38CV,ST39CV,ST40CV,ST43CV,ST44CV,ST45CV,ST46CV,...,ST35TA,ST36TA,ST38TA,ST39TA,ST40TA,ST43TA,ST44TA,ST45TA,ST46TA,ST47TA,ST48TA,ST49TA,ST50TA,ST51TA,ST52TA,ST54TA,ST55TA,ST56TA,ST57TA,ST58TA,ST59TA,ST60TA,ST62TA,ST64TA,ST72TA,ST73TA,ST74TA,ST82TA,ST83TA,ST84TA,ST85TA,ST90TA,ST91TA,ST93TA,ST94TA,ST95TA,ST97TA,ST98TA,ST99TA,intercept
2,3096,2191,3784,2344,2675,2063,8561,3208,14310,7538,1571,14405,20381,11549,9488,8848,2175,1109,6150.0,6085,5977,1828,1392,5996,3021,1562.0,489,8597,6703,8310,2281,11219,7499,7580,4451,7417,3979,2317.0,4316,1783,...,1.891,2.540,1.879,2.279,2.548,2.298,2.861,2.528,2.827,2.234,1.404,1.940,2.285,2.436,1.800,2.940,2.152,2.692,1.756,2.607,2.221,3.703,2.175,1.064,2.058,2.168,2.414,1.885,3.303,2.702,2.617,2.108,2.526,2.151,2.003,2.534,1.996,2.163,2.643,1
5,3540,1428,3918,2111,3539,1935,6955,3082,10875,8773,1814,15407,20977,11587,10094,9856,1799,836,7960.0,6815,6610,2549,1745,4919,2491,2458.0,682,7869,11311,11582,1899,10270,6044,5178,4267,11255,4448,2021.0,4152,1550,...,1.843,2.503,1.696,2.468,2.598,2.156,2.060,2.341,2.497,2.118,1.551,1.653,2.310,2.052,2.068,2.820,2.080,2.431,1.816,2.443,2.084,3.491,1.776,0.999,2.357,2.339,2.195,1.504,3.472,2.927,2.255,2.359,2.828,1.972,1.868,2.326,1.608,2.173,2.861,1
7,3960,1412,3344,1360,2559,1753,7071,2739,8755,6926,1539,11174,14527,12024,8034,6700,1587,513,5516.0,4430,4686,2075,1725,5026,1807,1953.0,360,6680,10128,7727,1703,9640,4919,5768,2611,8757,2619,1365.0,3087,1319,...,2.026,2.437,1.794,2.021,2.732,1.972,1.999,2.300,2.438,2.056,1.294,1.682,2.162,2.131,2.105,2.734,1.893,2.382,2.142,2.363,2.303,3.901,1.488,0.839,2.220,2.031,2.437,1.775,3.719,2.673,2.438,2.278,2.659,2.047,2.156,2.275,1.677,2.237,2.673,1
8,3421,2267,3583,2446,2149,2145,10472,2898,12527,7824,1715,12845,20170,10060,10947,10244,1705,681,7221.0,6480,6282,2096,1558,4822,2957,1910.0,634,7944,9090,10953,2415,9383,6334,6316,4632,8198,3528,2217.0,2754,1603,...,1.904,2.565,1.752,2.483,2.499,2.058,2.551,2.012,2.434,2.119,1.568,1.827,2.552,2.324,1.840,3.020,1.936,2.417,1.850,2.495,2.009,3.378,2.228,1.052,2.334,2.615,2.269,1.766,3.743,2.301,2.683,2.172,2.759,2.671,1.979,2.511,1.921,2.297,2.752,1
9,3506,1711,4559,2093,2828,2126,7977,3209,11138,8110,1909,12931,18123,11025,9640,8837,1800,688,5591.0,5575,5371,2077,2503,5398,1957,1146.0,661,7996,9165,11073,1986,9697,6648,5443,3077,8401,3050,2041.0,4047,1738,...,1.983,2.633,1.813,2.335,2.636,2.159,2.853,2.611,3.139,2.480,1.478,1.895,2.282,2.176,2.227,2.956,2.538,2.687,1.958,2.700,2.474,3.170,1.921,1.106,2.530,3.209,2.618,1.804,2.903,3.270,2.789,2.413,3.045,2.874,2.139,2.822,1.926,2.624,2.894,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,2873,1405,4460,1500,3826,2115,7288,3502,10433,8674,1585,13551,17946,12005,9445,7897,2318,1031,7836.0,6132,6187,3415,1742,6014,2759,2407.0,527,9149,10233,8581,2355,10320,6568,5994,3626,9445,3229,1981.0,4969,1612,...,2.148,2.770,1.859,2.407,2.697,2.459,1.940,2.486,2.768,2.404,1.667,1.942,2.703,2.467,2.278,3.272,2.390,2.773,2.237,2.797,2.378,4.402,2.287,1.282,2.331,2.457,2.563,1.812,4.129,3.035,2.854,2.435,2.773,2.451,2.171,2.869,1.781,2.508,2.739,1
1588,3652,1737,3307,1624,2904,1286,7389,3073,10720,7324,1957,9254,17188,9818,7836,7252,1685,601,5985.0,4861,4931,1961,1039,5042,1875,1712.0,501,7438,10176,7597,2065,8664,5971,5617,3366,9233,3231,1699.0,3261,1989,...,2.101,2.671,1.861,2.338,2.789,2.324,2.190,2.400,2.754,2.373,1.318,2.091,2.715,2.518,2.156,2.873,2.238,2.774,2.148,2.423,2.488,3.455,2.419,1.039,2.613

## Train Test Split for Second Task

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y2['ADAS11'], test_size = 0.2, shuffle = True, random_state = 27)
X_train = X_train.drop(labels='intercept',axis=1)
X_test = X_test.drop(labels='intercept',axis=1)


In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print(X_train.shape[1])

140


## Standarization

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(X_train)
X_test_minmax = mm_scaler.fit_transform(X_test)

In [ ]:
scaler = StandardScaler()
X_train_stnd = scaler.fit_transform(X_train)
X_test_stnd = scaler.fit_transform(X_test)

## Prediction of ADAS11

In [ ]:
linmodel = LinearRegression()

In [ ]:
linmodel.fit(X_train_stnd, y_train)
pred = linmodel.predict(X_test_stnd)
np.sqrt(mean_squared_error(y_test, pred))

5.498142758062902

In [ ]:
r2_score(y_test, pred)

-0.040100694566048034

## XGBoost Regressor

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0.1,
    max_depth=1
)
regressor.fit(X_train_stnd, y_train)
y_pred = regressor.predict(X_train_stnd)

[14:43:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_score(y_train, y_pred)

0.5619531974322569

## KNN Regressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train_stnd, y_train)
pred = neigh.predict(X_test_stnd)

In [ ]:
r2_score(y_test, pred)

0.04107170625663614

## Prediction of ADAS13

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y2['ADAS13'], test_size = 0.2, shuffle = True, random_state = 27)
X_train = X_train.drop(labels='intercept',axis=1)
X_test = X_test.drop(labels='intercept',axis=1)

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [ ]:
scaler = StandardScaler()
X_train_stnd = scaler.fit_transform(X_train)
X_test_stnd = scaler.fit_transform(X_test)

In [ ]:
linmodel2 = LinearRegression()

In [ ]:
linmodel2.fit(X_train_stnd, y_train)
pred = linmodel2.predict(X_test_stnd)
np.sqrt(mean_squared_error(y_test, pred))

8.144211126613488

In [ ]:
r2_score(y_test, pred)

0.03972928634900663

## XGBoost Regressor

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0.1,
    max_depth=1
)
regressor.fit(X_train_stnd, y_train)
y_pred = regressor.predict(X_train_stnd)

[14:43:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_score(y_train, y_pred)

0.585916787948386

## KNN Regressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train_stnd, y_train)
pred = neigh.predict(X_test_stnd)

In [ ]:
r2_score(y_test, pred)

0.07357289830522662

## Prediction of MMSE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y2['MMSE'], test_size = 0.2, shuffle = True, random_state = 27)
X_train = X_train.drop(labels='intercept',axis=1)
X_test = X_test.drop(labels='intercept',axis=1)

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [ ]:
scaler = StandardScaler()
X_train_stnd = scaler.fit_transform(X_train)
X_test_stnd = scaler.fit_transform(X_test)

In [ ]:
linmodel3 = LinearRegression()
linmodel3.fit(X_train_stnd, y_train)
pred = linmodel3.predict(X_test_stnd)
np.sqrt(mean_squared_error(y_test, pred))

2.390168189216047

In [ ]:
r2_score(y_test, pred)

-0.03363444767156931

## XGBoost Regressor

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0.1,
    max_depth=1
)
regressor.fit(X_train_stnd, y_train)
y_pred = regressor.predict(X_train_stnd)

[14:43:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_score(y_train, y_pred)

0.5257880539021998

## KNN Regressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train_stnd, y_train)
pred = neigh.predict(X_train_stnd)

In [ ]:
r2_score(y_train, pred)

0.4858278878084398

## Analysis
In this task the preiction of the following features has been acheived:

1.   Alzheimer's Disease Assessment Scale (11 questions version)
2.   Alzheimer's Disease Assessment Scale (13 questions version)
3.   Mini-Mental State Examination

The models used in the analysis are the following:
<ol>
  <li>Linear Regression</li>
  <li>XGBoost Regression</li>
  <li>KNN Regression</li>
</ol>

For the first assessment:

The r2 scores was bad for linear regression; the higher was XGBoost with 0.56 in training data and 0.34 in test data

For the second assessment:

The higher model was XGBoost with 0.59 in training data and 0.36 in test data.

For the third assessment:

In training data the linear regression performance is slightly better in r2 

values. However, in test data the XGBoost is better.

***NOTE: The r2 scores are negative in some models which means that the models predictions is worse than the constant line. This implies that the fitting of the data was bad. I have applied standarization to help in fitting the data which made the r2 scores better. Yet, it is not the best accuracy out there.***





## Build MultiOutputRegressor

Since now we know XGBoost is the best model for predicting all three scores, let's combine into one the three XGBoost models I used above to predict each model one by one. To do this, we need to use MultiOutputRegressor.

In [ ]:
X_train_task2, X_test_task2, y_train_task2, y_test_task2 = train_test_split(X, Y2, test_size = 0.2, shuffle = True, random_state = 27)
X = X.drop(labels='intercept',axis=1)
X_train_task2 = X_train_task2.drop(labels='intercept',axis=1)
X_test_task2 = X_test_task2.drop(labels='intercept',axis=1)

X = np.asarray(X)
Y2 = np.asarray(Y2)

In [ ]:
X_train_task2 = np.asarray(X_train_task2)
y_train_task2 = np.asarray(y_train_task2)

X_test_task2 = np.asarray(X_test_task2)
y_test_task2 = np.asarray(y_test_task2)

In [ ]:
scaler_task2 = StandardScaler()
X_train_stnd_task2 = scaler_task2.fit_transform(X_train_task2)
X_test_stnd_task2 = scaler_task2.fit_transform(X_test_task2)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

regressor_task2 = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0.1,
    max_depth=1
)

wrapper = MultiOutputRegressor(regressor_task2)

cv = KFold(n_splits=10)
n_scores = cross_val_score(wrapper, X, Y2, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# absolute values
n_scores = abs(n_scores)
# summarize performance
print("(mean, standard deviation)")
print((np.mean(n_scores), np.std(n_scores)))

wrapper.fit(X, Y2)
# wrapper.predict(X_test_stnd_task2)

(mean, standard deviation)
(3.631646951393955, 0.22365756555015046)
[14:43:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0.1,
                                            importance_type='gain',
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=1, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='reg:linear',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verb

### Done! Saving the multi-output XGBoost model now

In [ ]:
dump(wrapper, 'reg_model.joblib')

['reg_model.joblib']